# Coding Quiz 12

In [12]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm


In [4]:
data = pd.read_csv("homework_12.1.csv", index_col=0)
data.head()

,Y,Time,Group
0,-0.232900,-1.193204,0
1,2.848846,-1.607748,1
2,0.550209,-0.269793,0
3,2.198280,7.743730,0
4,4.111044,-4.244359,1


In [7]:
event_time = 0
treated = data["Group"] == 1
control = data["Group"] == 0

data["post_event"] = (data["Time"] > event_time).astype(int)
data["treated"] = treated.astype(int)
data["value"] = data["Y"]
model = smf.ols("value ~ treated * post_event", data=data).fit()

model.params

Intercept             0.980217
treated               2.042010
post_event            1.512989
treated:post_event    0.944010
dtype: float64

In [8]:
data = pd.read_csv("homework_12.2.csv", index_col=0)
data.head()

,Y,Time,Group
0,0.283614,-3.614416,0
1,3.181495,-3.064196,0
2,1.401619,-4.109261,0
3,2.019632,-0.706696,1
4,1.032628,-1.806227,0


In [27]:
# keep only pre-period and make a copy
before = data[data["Time"] < 0].copy()

# interaction term: Group * Time
before["time_group"] = before["Group"] * before["Time"]

# features (X) and outcome (y)
X = before[["Group", "Time", "time_group"]]
X = sm.add_constant(X)

y = before["Y"]

# run OLS
results = sm.OLS(y, X).fit()

print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.324
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     1591.
Date:                Tue, 02 Dec 2025   Prob (F-statistic):               0.00
Time:                        10:50:43   Log-Likelihood:                -13981.
No. Observations:                9944   AIC:                         2.797e+04
Df Residuals:                    9940   BIC:                         2.800e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9996      0.022     45.154      0.0

In [29]:
np.random.seed(0) 

X1 = np.random.normal(0, 1, 1000) 

X2 = np.random.normal(0, 1, 1000) + X1 

X3 = np.random.normal(0, 1, 1000) + 2 * X2 

variance_inflation_factors = {}
for i, col in enumerate([X1, X2, X3]):
    X = np.column_stack([c for j, c in enumerate([X1, X2, X3]) if j != i])
    X = sm.add_constant(X)
    model = sm.OLS(col, X).fit()
    r_squared = model.rsquared
    vif = 1 / (1 - r_squared)
    variance_inflation_factors[f'X{i+1}'] = vif
print(variance_inflation_factors)

{'X1': np.float64(1.9783742480901259), 'X2': np.float64(10.108728682206545), 'X3': np.float64(8.989002191063056)}
